In [ ]:
#در این پروژه از نوت بوک کگل استفاده می کنیم (به دلیل استفاده از tensorflow & Keras)
#این دیتاست توسط یک یوزر آپلود شده است
import numpy as np 
import pandas as pd 

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


لایبرری های مورد نیاز 

In [ ]:
#importing libraries
import matplotlib.pyplot as plt
from keras import Model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization, concatenate
from tensorflow.keras import Input

ابتدا کمی در باره ی دیتاست مطالعه کنیم:

In [ ]:
df=pd.read_csv('/kaggle/input/house-prices-and-images-socal/socal2.csv')
print(df.head())

In [ ]:
X_features=df[['n_citi','bed','bath','sqft','price']]


print(X_features)
print(X_features.shape)

با توجه به ویژگی های موجود ویژگی ها را نرمال می کنیم

In [ ]:


bm=max(X_features['bed'])
sqftm=max(X_features['sqft'])
pricem=max(X_features['price'])
bathm=max(X_features['bath'])
citim=max(X_features['n_citi'])
X_features['n_citi']=X_features['n_citi']/citim
X_features['bed']=X_features['bed']/bm
X_features['sqft']=X_features['sqft']/sqftm
X_features['bath']=X_features['bath']/bathm
X_features['price']=X_features['price']/pricem

> سایز عکس ها متفاوت است و این برای ترینینگ مشکل ساز است بنابراین به ۶۴در۶۴ تغییر می دهیم همه را و همزمان عکس ها را می خوانیم
در آرایه(تنسور)
X_images


In [ ]:
import os
import cv2
cnt=0
images_path='../input/house-prices-and-images-socal/socal2/socal_pics'
X_images=np.zeros((15474,64,64,3),dtype='uint32')
for i in range(15474):

    sample=cv2.imread(images_path+'/'+str(i)+'.jpg')
    imgs=cv2.resize(sample,(64,64))
    X_images[cnt]=imgs
    cnt+=1

print(cnt)

In [ ]:
X_images=X_images/255.0   #باید ری-اسکیل کنیم

مدل اصلی ما از ترکیب دو شبکه به وجود می آید:
شبکه ۱: در این قسمت ویژگی های غیر تصویری را به عنوان ورودی می دهیم و بعد از دو لایه ۵ ویژگی از آن استخراج می کنیم.

شبکه ۲: در این قسمت ویژگی های تصویری را با اسنفاده از یک شبکه cnn استخراج می کنیم

در آخر مجموعه ویژگی های استخراج شده را با هم به عنوان ورودی به شبکه پیش بینی کننده می دهیم و آن را ترین میکنیم.

دقت کنید که فاز ترینینگ هر ۳ یکی است و هزینه برای آپدیت هر ۳ استفاده می شود.

In [ ]:
def create_ann(dim, regress=False):
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation="relu"))
    model.add(Dense(4, activation="relu"))
    model.add(Dense(5,activation="sigmoid"))
    
    return model

In [ ]:
def create_cnn(height, width, depth, filters=(16, 32, 64), regress=False):
   
    inputShape = (height, width, depth)
    chanDim = -1
    
    inputs = Input(shape=inputShape)
    x = Conv2D(16, (3, 3), padding="same")(inputs)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(32, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(64, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Flatten()(x)
    x = Dense(16)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(0.5)(x)

    x = Dense(4)(x)
    x = Activation("relu")(x)
    model = Model(inputs, x)
    return model

## Splitting into training and test set

In [ ]:
# X_house_attributes -> continuous variables and price (output variable)
# X_house_images -> images
from sklearn.model_selection import train_test_split
split = train_test_split(X_features, X_images, test_size=0.25, random_state=42)
(Xatt_train,Xatt_test,Ximage_train,Ximage_test) = split

y_train , y_test = Xatt_train['price'].values , Xatt_test['price'].values

X1_train=Xatt_train[['n_citi','bed','bath','sqft']].values
X2_train=Ximage_train
X1_test=Xatt_test[['n_citi','bed','bath','sqft']].values
X2_test=Ximage_test

print(X1_train.shape)
print(X1_test.shape)
print(X2_train.shape)
print(X2_test.shape)
print(y_train.shape)
print(y_test.shape)

مدل نهایی

In [ ]:

mlp = create_ann(X1_train.shape[1], regress=False)
cnn = create_cnn(64, 64, 3, regress=False)
#ترکیب دو خروجی به عنوان ورودی
combinedInput = concatenate([mlp.output, cnn.output])

x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)



In [ ]:
from keras.optimizers import Adam
model = Model(inputs=[mlp.input, cnn.input], outputs=x)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mse", optimizer=opt)
# train the model


model.fit(x=[X1_train,X2_train], y=y_train,validation_data=([X1_test, X2_test], y_test),epochs=50, batch_size=64)


تست روی دیتای جدید

In [ ]:
attr_sample=df.loc[df['image_id'] == 877]
print(attr_sample)

In [ ]:
image_sample=cv2.imread('/kaggle/input/house-prices-and-images-socal/socal2/socal_pics/877.jpg')
sample_resized=cv2.resize(image_sample,(64,64))
plt.imshow(sample_resized)

Preprocessing the data so that the trained model can predict

In [ ]:
X1_final=np.zeros(4,dtype='float32')
X1_final[0]=attr_sample['n_citi']/citim
X1_final[1]=attr_sample['bed']/bm
X1_final[2]=attr_sample['bath']/bathm
X1_final[3]=attr_sample['sqft']/sqftm
y_ground_truth=attr_sample['price']

In [ ]:
X2_final=sample_resized/255.0

In [ ]:
print(X1_final.shape," ",X2_final.shape)

In [ ]:
y_pred=model.predict([np.reshape(X1_final,(1,4)),np.reshape(X2_final,(1,64,64,3))])

In [ ]:
print("Actual price: ",attr_sample['price'].values)
print("Predicted price: ",y_pred*pricem)